# ELYZA-tasks-100 による性能評価

In [ ]:
#!pip install accelerate autoawq datasets jsonlines sentencepiece
!pip install accelerate datasets jsonlines sentencepiece
# https://github.com/oobabooga/text-generation-webui/issues/4517#issuecomment-1815279467
!pip install https://github.com/casper-hansen/AutoAWQ/releases/download/v0.1.7/autoawq-0.1.7+cu118-cp310-cp310-linux_x86_64.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.0/154.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.5/227.5 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━

## ELYZA-tasks-100 データセットのダウンロード

モデルのダウンロードには 🤗datasets ライブラリを用いる。

データは `/content/dataset.jsonl` に保存する。

In [ ]:
import json
from pathlib import Path

from datasets import load_dataset

# Load the dataset
ds = load_dataset("elyza/ELYZA-tasks-100")

# Function to convert dataset to JSONL format and print
def dataset_to_jsonl(dataset, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        for entry in dataset:
            # Construct JSON object
            json_obj = {
                "input_text": entry['input'],
                "output_text": entry['output'],
                "eval_aspect": entry['eval_aspect']
            }

            # Write JSON object to file in JSONL format
            json_str = json.dumps(json_obj, ensure_ascii=False)
            file.write(json_str + '\n')

# Convert and write the dataset to a file in JSONL format
path_jsonl = Path("/content/dataset.jsonl")

if (path_jsonl.parent is not None) and (not path_jsonl.parent.exists()):
    path_jsonl.parent.mkdir(parents=True, exist_ok=True)

dataset_to_jsonl(ds["test"], path_jsonl)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

## 評価

ELYZA-tasks-100 データセットを用いて、指定したモデルの評価を実行する。

### プロンプトについて

モデルに回答を生成させるときのプロンプトは、各モデルのモデルカード (🤗HuggingFace の README ページ) に記載されているプロンプトを使用する。

> 📒 **ノート**
>
> この方法は、質問を直接入力する場合や、全モデルで共通のプロンプトを用いる場合に比べて公平性が劣る。
> しかし、実利用時は各モデルが最もよい性能を発揮するように、各モデルのプロンプトをチューニングすることが想定される。
> モデルカード記載のプロンプトをチューニング済みのプロンプトと見なすことで、実利用時の性能を比較することを狙いとする。

### ハイパーパラメータについて

ハイパーパラメータは 🤗HuggingFace の Code Snippet のとおりとする（公開者が指定したハイパーパラメータを、チューン済みとみなす）。
ただし、長文の回答を要求する設問もあるため、`max_new_tokens` は `1024` に統一する。

In [ ]:
import json
import jsonlines
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Setup model and tokenizer
model_name = "01-ai/Yi-6B-Chat"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype='auto',
    device_map="auto"
).eval()

def format_prompt(input_text):
    # ✋プロンプトは、モデルに合わせて調整（HuggingFace のモデルカード参照）
    messages = [{"role": "user", "content": input_text}]
    input_ids = tokenizer.apply_chat_template(
        conversation=messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors='pt'
    )
    return input_ids

def generate_text(input_ids):
    # Set seed for reproducibility
    seed = 42
    torch.manual_seed(seed)

    # ✋Hparamsは、モデルに合わせて調整（HuggingFace のモデルカード参照）
    output_ids = model.generate(
        input_ids.to('cuda'),
        max_new_tokens=1024,
    )

    response = tokenizer.decode(output_ids[0][input_ids.shape[1]:], skip_special_tokens=True)
    return response

def process_dataset(input_file, output_file):
    # Read dataset and generate predictions
    with jsonlines.open(input_file) as reader, jsonlines.open(output_file, mode='w') as writer:
        for obj in reader:
            prompt = obj['input_text']
            input_ids = format_prompt(prompt)
            generated_text = generate_text(input_ids)

            print(f"==============================")
            print(f"Q. {prompt}")
            print(f"A. {generated_text}")
            print(f"")

            writer.write({"pred": generated_text})

# Process the dataset
input_dataset = '/content/dataset.jsonl'
output_predictions = '/content/preds.jsonl'
process_dataset(input_dataset, output_predictions)

tokenizer_config.json:   0%|          | 0.00/1.90k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/2.21G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

Q. 仕事の熱意を取り戻すためのアイデアを5つ挙げてください。
A. 1. 休憩を取る: 仕事に疲れが積み重なっている場合、一晩の休憩を取ることで精神と体力を回復させることができます。
2. スポーツや体力活動をする: スポーツや体力活動を通じて身体と精神を刺激させ、仕事の熱意を再び取り戻せることができます。
3. 友人や家族と会う: 社会的相互作用が精神的にも役に立つことが分かっています。友人や家族と会うことで、仕事の熱意を取り戻すことができます。
4. 休暇を取る: 休暇を取って、自己回復へと移行し、仕事の熱意を再び取り戻せることができます。
5. 本を読む: 本を読むことで、新しいアイデアや考え方を学び、仕事の熱意を再び取り戻せます。

Q. クマが海辺に行ってアザラシと友達になり、最終的には家に帰るというプロットの短編小説を書いてください。
A. 海の渓谷の一隅に、クマがひび割れた森の中を探していた。彼女は家を出ると、そこから通りを歩いて森に入り、そこから海辺まで歩いていく。しかし、今度はその森の中で、そして海辺の砂の上で、アザラシと出会った。

アザラシは海辺の砂にいて、波打ちに楽しみにしていた。彼女はその姿を窺い、そしてその顔を見ると、その目には笑顔が浮かび、彼女には驚きだった。二人はすぐに相手を喜ばしく見て、森の中へと歩き出した。

森の中は、夏の日光が照らしていて、青い空と緑の森が美しくなっていた。アザラシはクマの顔を見つめながら、「ここはいい場所だと思うよ。森の中は静かで、海辺は音楽を聴くのにいい場所だ」と言った。クマはそれに同意し、二人は森の中で遊んだり、海辺の砂についてずっと立ち並べていたアザラシの姿を見て、笑った。

しばらくして、夜が訪れ、クマとアザラシは家に帰っていく。家の中は暖かく、明かりが点灯していて、そこから森の中や海辺の砂の上のことを思い出した。二人はこの後も、毎日そこから家に帰り、森と海辺で遊ぶことにした。

クマとアザラシの仲は、日々と深くなり、そして、彼らの心の中には、その日々の遊びと、海辺の砂の上でのお互いの笑顔が、永遠に詠んでいるように感じられた。

Q. 古代ギリシャを学ぶ上で知っておくべきポイントは？
A. 古代ギリシャを学ぶ上で知っておくべきポイントは多くありますが、基本的には以下のポイントがお薦めされます。

1

In [ ]:
import json
import jsonlines
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Setup model and tokenizer
model_name = "01-ai/Yi-6B-Chat"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype='auto',
    device_map="auto"
).eval()

def format_prompt(input_text):
    # ✋プロンプトは、モデルに合わせて調整（HuggingFace のモデルカード参照）
    messages = [{"role": "user", "content": input_text}]
    input_ids = tokenizer.apply_chat_template(
        conversation=messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors='pt'
    )
    return input_ids

def generate_text(input_ids):
    # Set seed for reproducibility
    seed = 42
    torch.manual_seed(seed)

    # ✋Hparamsは、モデルに合わせて調整（HuggingFace のモデルカード参照）
    output_ids = model.generate(
        input_ids.to('cuda'),
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.8,
    )

    response = tokenizer.decode(output_ids[0][input_ids.shape[1]:], skip_special_tokens=True)
    return response

def process_dataset(input_file, output_file):
    # Read dataset and generate predictions
    with jsonlines.open(input_file) as reader, jsonlines.open(output_file, mode='w') as writer:
        for obj in reader:
            prompt = obj['input_text']
            input_ids = format_prompt(prompt)
            generated_text = generate_text(input_ids)

            print(f"==============================")
            print(f"Q. {prompt}")
            print(f"A. {generated_text}")
            print(f"")

            writer.write({"pred": generated_text})

# Process the dataset
input_dataset = '/content/dataset.jsonl'
output_predictions = '/content/preds_sampled.jsonl'
process_dataset(input_dataset, output_predictions)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Q. 仕事の熱意を取り戻すためのアイデアを5つ挙げてください。
A. 1. 休憩を取る: 仕事の熱意を取り戻すために、定期的に休憩を取ることが重要です。これは、体力を回復し、精神を整理するためのものです。

2. 目標を設定する: 仕事の熱意を取り戻すために、具体的な目標を設定することが重要です。目標は、経験した成果を表すための指針ですし、進捗状況を把握するための基準です。

3. 効率的な時間管理: 仕事の熱意を取り戻すために、効率的な時間管理が必要です。これは、時間を切り抜くことで、重要な仕事への集中力を保つことができます。

4. 学習と向上の投資: 仕事の熱意を取り戻すために、学習と向上の投資をすることが重要です。これは、新しいアイデアや経験を得ることで、仕事の熱意を高めることができます。

5. パートナーとのチームワーク: 仕事の熱意を取り戻すために、パートナーとのチームワークを実行することが重要です。これは、ストレスを軽減し、新しいアイデアや解決策を生み出すことができます。

Q. クマが海辺に行ってアザラシと友達になり、最終的には家に帰るというプロットの短編小説を書いてください。
A. 海の渓谷の彼方、砂の街にある小さな家の前には、クマが海辺から歩いて帰ってきた。彼女はずっとアザラシとおしゃれな海辺を歩いてきたが、今回はそれほど気に入っていなかった。彼女は家の中でアザラシと出かけていく。

アザラシは海辺からクマの帰りを待っていた。彼女が家の前で歩いてくると、彼はようやく立ち上がり、彼女のすぐに傍に近づいてくる。クマはアザラシの顔を見ると、彼女の顔には海の黒い波がついていた。アザラシはクマの顔を笑顔で見つめていると、彼女が手を伸ばしてアザラシの頬をつかんでくる。

「お帰り、クマ」

アザラシはクマの手を引っ張り、彼女を家の中に連れて行く。家の中は静かで、アザラシのすぐ傍に座るクマの顔はすばらしい。彼女はアザラシのすぐ傍に座ると、彼女の髪の毛がアザラシの顔にかよってくる。クマはアザラシの顔をつつんでいると、彼女が手を伸ばしてアザラシの顎に触れる。

「これからも、アザラシと一緒に遊ぶようにしてくれ」

アザラシはクマの手を引っ張り、彼女を家の中の床に落とした。彼女はすぐに立ち上がり、彼女の顔にはついた海の黒い波が消えていく。アザラシはクマの背中を見て